In [1]:
int2op = {"00": "STOP",
"01": "ADD",
"02": "MUL",
"03": "SUB",
"04": "DIV",
"05": "SDIV",
"06": "MOD",
"07":"SMOD",
"08":"ADDMOD",
"09":"MULMOD",
"0a":"EXP",
"0b":"SIGNEXTEND",
"10":"LT",
"11":"GT",
"12":"SLT",
"13":"SGT",
"14":"EQ",
"15":"ISZERO",
"16":"AND",
"17":"OR",
"18":"XOR",
"19":"NOT",
"1a":"BYTE",
"1b":"SHL",
"1c": "SHR",
"1d": "SAR",
"20":"SHA3",
"30":"ADDRESS",
"31":"BALANCE",
"32":"ORIGIN",
"33":"CALLER",
"34":"CALLVALUE",
"35":"CALLDATALOAD",
"36":"CALLDATASIZE",
"37":"CALLDATACOPY",
"38":"CODESIZE",
"39":"CODECOPY",
"3a":"GASPRICE",
"3b":"EXTCODESIZE",
"3c":"EXTCODECOPY",
"3d":"RETURNDATASIZE",
"3e":"RETURNDATACOPY",
"3f": "EXTCODEHASH",
"40":"BLOCKHASH",
"41":"COINBASE",
"42":"TIMESTAMP",
"43":"NUMBER",
"44":"DIFFICULTY",
"45":"GASLIMIT",
"46": "CHAINID",
"47": "SELFBALANCE",
"50":"POP",
"51":"MLOAD",
"52":"MSTORE",
"53":"MSTORE8",
"54":"SLOAD",
"55":"SSTORE",
"56":"JUMP",
"57":"JUMPI",
"58":"PC",
"59":"MSIZE",
"5a":"GAS",
"5b":"JUMPDEST",
"60":"PUSH1",
"61":"PUSH2",
"62":"PUSH3",
"63":"PUSH4",
"64":"PUSH5",
"65":"PUSH6",
"66":"PUSH7",
"67":"PUSH8",
"68":"PUSH9",
"69":"PUSH10",
"6a":"PUSH11",
"6b":"PUSH12",
"6c":"PUSH13",
"6d":"PUSH14",
"6e":"PUSH15",
"6f":"PUSH16",
"70":"PUSH17",
"71":"PUSH18",
"72":"PUSH19",
"73":"PUSH20",
"74":"PUSH21",
"75":"PUSH22",
"76":"PUSH23",
"77":"PUSH24",
"78":"PUSH25",
"79":"PUSH26",
"7a":"PUSH27",
"7b":"PUSH28",
"7c":"PUSH29",
"7d":"PUSH30",
"7e":"PUSH31",
"7f":"PUSH32",
"80":"DUP1",
"81":"DUP2",
"82":"DUP3",
"83":"DUP4",
"84":"DUP5",
"85":"DUP6",
"86":"DUP7",
"87":"DUP8",
"88":"DUP9",
"89":"DUP10",
"8a":"DUP11",
"8b":"DUP12",
"8c":"DUP13",
"8d":"DUP14",
"8e":"DUP15",
"8f":"DUP16",
"90":"SWAP1",
"91":"SWAP2",
"92":"SWAP3",
"93":"SWAP4",
"94":"SWAP5",
"95":"SWAP6",
"96":"SWAP7",
"97":"SWAP8",
"98":"SWAP9",
"99":"SWAP10",
"9a":"SWAP11",
"9b":"SWAP12",
"9c":"SWAP13",
"9d":"SWAP14",
"9e":"SWAP15",
"9f":"SWAP16",
"a0":"LOG0",
"a1":"LOG1",
"a2":"LOG2",
"a3":"LOG3",
"a4":"LOG4",
"b0":"PUSH",
"b1":"DUP",
"b2":"SWAP",
"f0":"CREATE",
"f1":"CALL",
"f2":"CALLCODE",
"f3":"RETURN",
"f4":"DELEGATECALL",
"f5":"CREATE2",
"fa":"STATICCALL",
"fd":"REVERT",
"fe":"INVALID",
"ff":"SELFDESTRUCT"}

In [ ]:
from datasets import Dataset
import pandas as pd
import re

In [ ]:
data = pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/dataset_example/labeled_SBW_datasets.pkl")
data

In [ ]:
data = data[['source_code']]

In [ ]:
def compiler(input):
    pass

In [ ]:
data.source_code = data.source_code.apply(compiler)

In [ ]:
data = Dataset.from_pandas(data)

In [ ]:
def get_training_corpus():
    return (
        data[i : i + 1000]["source_code"]
        for i in range(0, len(data), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
example = '''pragma solidity ^0.8.17;

contract HelloWorld {
    string public greet = "Hello World!";
}'''

tokens = old_tokenizer.tokenize(example)
tokens

In [ ]:
old_tokenizer.add_tokens(list(int2op.values()))

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [ ]:
tokens = tokenizer.tokenize(example)
tokens

In [ ]:
tokenizer.save_pretrained("/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/")

# Pretrain

In [ ]:
data_train = Dataset.from_pandas(train)
data_test = Dataset.from_pandas(test)
data_val = Dataset.from_pandas(val)

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["source_code"]], padding = "max_length", truncation=True)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [ ]:
tokenized_data_train = data_train.map(preprocess_function, batched = True, num_proc=4, remove_columns = data_train.column_names)
tokenized_data_val = data_val.map(preprocess_function, batched = True, num_proc=4, remove_columns = data_val.column_names)


In [ ]:
lm_dataset_train = tokenized_data_train.map(group_texts, batched=True, num_proc=4 )
lm_dataset_val = tokenized_data_val.map(group_texts, batched=True, num_proc=4)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=new_tokenizer, mlm_probability=0.15)
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
print(model)
# %%
len(new_tokenizer)
## Resize 
model.resize_token_embeddings(len(new_tokenizer))
print(model)

In [ ]:
## Step 5: Setup training_args 
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="pre-train-mlm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10000,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train,
    eval_dataset=lm_dataset_val,
    data_collator=data_collator,
)


In [ ]:
trainer.train()


In [ ]:
trainer.save_model("/home/bombbom/Documents/NLP_in_Detection_System/save_model/pre_train_opcode/")